### Imports and Setup
Importing the necessary libraries and setting up the correct URL for the API.

In [ ]:
import requests
from pprint import pprint
import json 

URL = "http://127.0.0.1:8000/"

### Config
Parameters send to the server. Everytask you send will be processed with this confif JSON. Every parameter has a short description.

In [ ]:
data_to_send = {
    "config":
        {
        "QAOA":{
            "general":{ # General parameters in the QAOA algorithm
                "func_find_par": { # Name of the function in the QAOA algorithm
                    "method": "COBYLA", # Method to use in optimization
                    "tolerance": 0.001, # Tolerance of the optimization
                    "options": {"maxiter": 100000} # Options of the optimization
                },
                "func_find_best_parameters": {
                    "pool_size": 10, # Number of virtual processors to use
                    "iter_init": 100, # Number of solutions to generate for every P-value
                    "best_number": 15, # Number of best solutions to keep for the next P-value
                    "p_start": 3, # Starting P-value
                    "p_end": 10, # Ending P-value
                    "use_next_params": True, # Use the parameters learned in the previous P-value
                    "beta_corr_thr": 0.9, # Threshold for the beta correction to be accepted as good solution
                    "gamma_corr_thr": 0.9 # Threshold for the gamma correction to be accepted as good solution
                }
            },
            "jsp": {
                "max_time": 4, # Maximum time that the solution can take
                "problem_version": "optimization" # Version of the problem to solve (optimization or decizion)
            },
            "maxcut": {
                "n": -1, # Number of vertices in random graph (if -1, data taken from input) 
                #"a": 1
            }
        },
        
        "D-Wave": {
            "jsp":{
                "mode": "sim_pyqubo",
                "num_reads": 1000,
                "weight_one_hot": 3,
                "weight_precedence": 1,
                "weight_share":2
            }
        }
    }
}


### Data for problems
Provide input data for the problems. Sample data given. You can use this data to test your code or you can use your own. Try to use the same format (you will be notified if format is incorrect).

In [ ]:
data = {
    "ExactCover": {
        "routes": "[{1,4,7},{1,4},{4,5,7},{3,5,6},{2,3,6,7},{2,7}]"
    },
    "MaxCut": {
        "V": "[0,4,5,8,1,3,7,9,6,2]",
        "E": "[(0,4),(0,5),(0,8),(4,6),(4,7),(5,8),(5,1),(8,1),(1,9),(3,7),(3,2),(3,9),(7,6),(9,2),(6,2)]"
    },
    "MaxCut_Weighted": {
        "V": "[0,1,2,3,4]",
        "E": "[(2,4),(4,1),(1,0),(0,2),(0,3),(3,1)]",
        "W": "[4,9,5,5,6,2]"
    },
    "JSP": {
        "Tasks": "{'cupcakes':[('mixer',2) ,('oven',1)],'smoothie':[('mixer',1)],'lasagna':[('oven',2)]}"
    },
}

### Problems to solve
List of problems to solve. Place 1 if you want to solve this problem. Place 0 otherwise. You can send multiple problems to the server. Remember that all problems will have te same config. If you want to change the config for each problem you have to send those separately.

In [ ]:
Problems_to_solve = {
    "ExactCover": 0,
    "MaxCut": 1,
    "MaxCut_Weighted": 1,
    "JSP": 0
}

### Review data 
Here you can see the problems you want to solve and the data you have provided. Check if the data is correct and that you want to proceed. If you want to change something go back to previous cells.

In [ ]:
data_to_send["solve"] = {
    "input_data": [],
    "problems": [],}
for problem in Problems_to_solve:
    if Problems_to_solve[problem] == 1:
        data_to_send["solve"]["input_data"].append(data[problem])
        data_to_send["solve"]["problems"] += [problem]
print(f'Problems to solve: {data_to_send["solve"]["problems"]}')
print(f'Input data: {data_to_send["solve"]["input_data"]}')

#TODO: Visualize the input data


### Sending the data to the server
This cell sends the data to the server. It will notify you if the data is incorrect. Otherwise it will start the processing and return the ids of the tasks. Be sure to use this cell after you have reviewed the data in the previous one to ensure no server resources are wasted.

In [ ]:
if data_to_send["solve"]["problems"] == []:
    print("No problems to solve")
    exit()
ids = requests.post(f"{URL}solve/", json=data_to_send).json()
print(f'Solving problems with ids: {ids}')
data_to_send["solve"]["problems"] = []

### Getting the results
This cell will get the results of the tasks. It will notify you if the task is not finished yet. Otherwise it will return the results.

In [ ]:
for id in ids:
    print(f'Checking status of task with id: {id}')
    status = requests.get(f"{URL}result/{id}").json()
    if status["status"] == "Solved":
        print(f'Problem {status["problem"]} with id: {id} solved')
        print(f'Result: {status["solution"]}\n')
    else:
        print(f'Problem {status["problem"]} with id: {id} not solved yet\n')

#TODO: Visualize the result and save to file